<a href="https://colab.research.google.com/github/Santos-Rf/Projeto_01-Analise_de_dados_Olist-DSX/blob/main/DSX_Projeto_An%C3%A1lise_Explorat%C3%B3ria_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="color: #6c5b7b">Projeto 01 - Análise Exploratória de Dados</h1>

<p>Bem-vindos ao <strong>Data Science Experience</strong>! Nesse 1o projeto, vamos mostrar como realizar uma <strong>análise exploratória de dados</strong> da base de dados de e-commerce de uma grande empresa brasileira, a <strong>Olist</strong>.</p>

<p>A <strong>análise exploratória de dados</strong> é uma das principais técnicas utilizadas em <strong>Data Science</strong>, pois nos permite compreender melhor os dados e encontrar <em>insights</em> valiosos que podem ser usados em decisões de negócios. Neste notebook, vamos explorar a base de dados da Olist e responder algumas perguntas importantes sobre o comportamento dos consumidores no mercado de e-commerce.</p>

<p>Para isso, usaremos a linguagem <strong>Python</strong> e as bibliotecas <strong>Pandas</strong>, <strong>Numpy</strong> e <strong>Matplotlib</strong> para manipulação e visualização de dados. Além disso, utilizaremos a biblioteca <strong>Plotly</strong> para criar gráficos interativos e mais complexos, permitindo uma análise mais detalhada dos dados.</p>

<p>Ao final deste projeto, estaremos mais familiarizados com as técnicas utilizadas em análise exploratória de dados e teremos uma visão mais clara sobre como aplicá-las em projetos futuros.</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
customers_df = pd.read_csv("./data/raw/olist_customers_dataset.csv")
geo_df = pd.read_csv("./data/raw/olist_geolocation_dataset.csv")
orderitem_df = pd.read_csv("./data/raw/olist_order_items_dataset.csv")
orderpay_df = pd.read_csv("./data/raw/olist_order_payments_dataset.csv")
orderreviews_df = pd.read_csv("./data/raw/olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("./data/raw/olist_orders_dataset.csv")
products_df = pd.read_csv("./data/raw/olist_products_dataset.csv")
sellers_df = pd.read_csv("./data/raw/olist_sellers_dataset.csv")
categname_df = pd.read_csv("./data/raw/product_category_name_translation.csv")
pd.set_option('display.max_columns', 500)

In [ ]:
# Vendo as 10 linhas do arquivo de clientes
customers_df.head(10)

In [ ]:
# Quantas linhas e colunas tem o dataframe de clientes
customers_df.shape

In [ ]:
print("customers_df:")
display(customers_df.head())

print("\ngeo_df:")
display(geo_df.head())

print("\norderitem_df:")
display(orderitem_df.head())

print("\norderpay_df:")
display(orderpay_df.head())

print("\norderreviews_df:")
display(orderreviews_df.head())

print("\norders_df:")
display(orders_df.head())

print("\nproducts_df:")
display(products_df.head())

print("\nsellers_df:")
display(sellers_df.head())

print("\ncategname_df:")
display(categname_df.head())

In [ ]:
print("customers_df:\n")
display(customers_df.info())

print("\ngeo_df:\n")
display(geo_df.info())

print("\norderitem_df:\n")
display(orderitem_df.info())

print("\norderpay_df:\n")
display(orderpay_df.info())

print("\norderreviews_df:\n")
display(orderreviews_df.info())

print("\norders_df:\n")
display(orders_df.info())

print("\nproducts_df:\n")
display(products_df.info())

print("\nsellers_df:\n")
display(sellers_df.info())

print("\ncategname_df:\n")
display(categname_df.info())

In [ ]:
customers_df = customers_df.rename(columns={"customer_zip_code_prefix": "zip_code"})
geo_df = geo_df.rename(columns={"geolocation_zip_code_prefix": "zip_code"})

In [ ]:
customers_df.head()

In [ ]:
data = orders_df.merge(customers_df, on="customer_id").merge(orderitem_df, on="order_id").merge(products_df, on="product_id").merge(categname_df, on="product_category_name").merge(orderpay_df, on="order_id").merge(sellers_df, on="seller_id").merge(orderreviews_df, on="order_id")

In [ ]:
data.head()

Temos ids de pedidos duplicados. Isso se deve ao fato de que o mesmo pedido pode ser pago com vários métodos de pagamento diferentes.

In [ ]:
# Porcentagem de valores nulos
(100 * data.isna().sum() / len(data) ).sort_values(ascending=False)

In [ ]:
# Estatística descritiva básica
data.describe()

<h2 style="color: #6c5b7b">Como estão distribuídas as avaliações (notas)?</h2>



In [ ]:
fig = px.histogram(data, x="review_score")
fig.show()

In [ ]:
percentage = data["review_score"].value_counts(normalize=True) * 100

fig = px.bar(percentage, x=percentage.index, y=percentage.values, text=percentage.values, labels={"x": "Nota", "y": "Percentagem"})
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.show()


Mais de 75% dos clientes deram uma pontuação igual ou maior que 4. 12,5% deram uma pontuação de 1 e cerca de 12% deram uma pontuação de 3 ou 2.

<h2 style="color: #6c5b7b">Proporção de clientes gerando a maior parte da receita.</h2>


<p>A <strong>Análise de Pareto</strong> é uma técnica importante para identificar e priorizar os elementos mais relevantes em um conjunto de dados, permitindo a concentração de esforços nas áreas mais críticas e relevantes, maximizando resultados e minimizando custos. É uma ferramenta simples e de fácil aplicação.</p>

<p>É a famosa regra 20/80, 20% que representam 80% do resultado</p>

<p>Podemos utilizar para várias análises! Vamos começar com a proporção de clientes que mais gastam.</p>


In [ ]:
# Clientes com o maior número cumulativo de pedidos (em pagamentos).
top_customers = data.groupby("customer_unique_id")["payment_value"].sum().reset_index().sort_values("payment_value", ascending=False)
top_customers.rename(columns={"payment_value":"total_paid"}, inplace=True)

In [ ]:
# calcular as colunas "% of Total Sales" e "Cum % of Total Sales"
top_customers["% of Total Sales"] = (top_customers["total_paid"] / top_customers["total_paid"].sum()) * 100
top_customers["Cum % of Total Sales"] = top_customers["% of Total Sales"].cumsum()

In [ ]:
# criar um gráfico de linhas do Plotly
fig = px.line(top_customers, x=range(1, len(top_customers) + 1), y="Cum % of Total Sales")

# definir as etiquetas do eixo x e y e o título do gráfico
fig.update_layout(
    xaxis_title="Número de Clientes",
    yaxis_title="Total Vendas Cumulativo %",
    title="Contribuição % para as vendas por número de clientes"
)

# adicionar uma linha de preenchimento abaixo do gráfico
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=0,
    y0=0,
    x1=40000,
    y1=1,
    fillcolor="green",
    opacity=0.2,
    layer="below"
)

# atualizar o layout da forma para ajustar a altura do preenchimento
fig.update_shapes(dict(xref='x', yref='paper'))

# adicionar um texto explicativo na figura
fig.add_annotation(
    x=55000,
    y=75,
    text="40k clientes (+-42% do total)<br> representam +-80% das vendas",
    font=dict(
        size=14,
        color="black"
    ),
    showarrow=False,
)

# exibir a figura
fig.show()

<h2 style="color: #6c5b7b">Quem são os clientes que mais gastam?</h2>


Os resultados dessa análise podem gerar diversos insights para a área de marketing! Por exemplo:
- Monitorar os clientes que mais gastam-
- Oferecer promoções
- Tentar vender mais produtos
- Realizar entregas personalizadas




In [ ]:
# renomear a coluna "payment_value" para "total_paid"
top_customers.rename(columns={"payment_value" : "total_paid"}, inplace=True)

# criar um gráfico de barras do Plotly
fig = px.bar(top_customers[:10], x="total_paid", y="customer_unique_id", orientation="h")

# atualizar as configurações de layout do gráfico
fig.update_layout(
    title="Top 10 Clientes por Valor Total Pago",
    xaxis_title="Valor Total Pago",
    yaxis_title="ID do Cliente Único"
)

# exibir o gráfico
fig.show()


<h2 style="color: #6c5b7b">Principais cidades por número de pedidos por estado.</h2>


<p>Vamos verificar quais são as cidades que mais tem números de pedido!</p>



In [ ]:
# agrupar o dataframe por "customer_state" e contar o número de "order_id" por estado
top_orders_cities = data.groupby("customer_state")["order_id"].count().reset_index().sort_values("order_id", ascending=False)

# renomear a coluna "order_id" para "count"
top_orders_cities.rename(columns={"order_id":"count"}, inplace=True)

# criar um gráfico de barras do Plotly
fig = px.bar(top_orders_cities[:10], x="count", y="customer_state", orientation="h")

# atualizar as configurações de layout do gráfico
fig.update_layout(
    title="TOP 10 Estados por Número de Pedidos",
    xaxis_title="Número de Pedidos",
    yaxis_title="Estado"
)

# exibir o gráfico
fig.show()


<h2 style="color: #6c5b7b">Cidades com maior geração de receita.</h2>


<p>Agora podemos avaliar não a quantidade de pedidos mas o quanto de dinheiro cada cidade traz.</p>



In [ ]:
top_ordersbyvalue_cities = data.groupby("customer_city")["payment_value"].sum().reset_index().sort_values("payment_value", ascending=False)
top_ordersbyvalue_cities["% of Total Payments"] = (top_ordersbyvalue_cities["payment_value"] / top_ordersbyvalue_cities["payment_value"].sum()) * 100
top_ordersbyvalue_cities["Cum % of Total Payments"] = top_ordersbyvalue_cities["% of Total Payments"].cumsum()

In [ ]:
# criar um gráfico de barras do Plotly
fig = px.bar(top_ordersbyvalue_cities[:10], x="% of Total Payments", y="customer_city", orientation="h")

# atualizar as configurações de layout do gráfico
fig.update_layout(
    title="TOP 10 Cidades por Geração de Receita",
    xaxis_title="Porcentagem do Total de Pagamentos",
    yaxis_title="Cidade do Cliente"
)

# exibir o gráfico
fig.show()


In [ ]:
# criar um gráfico de linha do Plotly
fig = px.line(top_ordersbyvalue_cities, x=range(1, len(top_ordersbyvalue_cities)+1), y="Cum % of Total Payments")

# atualizar as configurações de layout do gráfico
fig.update_layout(
    title="% de Contribuição das Vendas por Número de Cidades",
    xaxis_title="Número de Cidades",
    yaxis_title="% de Contribuição para as Vendas"
)

# preencher a área abaixo da curva
fig.add_shape(
    type="rect",
    xref="x",
    yref="y",
    x0=0,
    y0=0,
    x1=358,
    y1=top_ordersbyvalue_cities["Cum % of Total Payments"][357],
    fillcolor="green",
    opacity=0.3,
    layer="below",
    line_width=0
)

# adicionar um texto ao gráfico
fig.add_annotation(
    x=1000,
    y=70,
    text="358 cidades (+-8,7% do total) <br>contribuem para +-80% das vendas.",
    font=dict(
        size=14,
        color="black"
    ),
    showarrow=False
)

# exibir o gráfico
fig.show()


In [ ]:
print("Número de cidades que contribuem com 80% das vendas totais:",
      len(top_ordersbyvalue_cities[top_ordersbyvalue_cities["Cum % of Total Payments"] <= 80]),
      "ou em in %:",
      (len(top_ordersbyvalue_cities[top_ordersbyvalue_cities["Cum % of Total Payments"] <= 80]) / len(top_ordersbyvalue_cities)) * 100)

<h2 style="color: #6c5b7b">Como os pedidos variam ao longo do tempo?</h2>


<p>Como os pedidos são feitos ao longo do dia? De manhã? De tarde? Na madruga? Às segundas, sábados? É importante saber se estamos vendendo mais ou menos, nos horários e dias das semana!</p>



In [ ]:
# Total de pedidos por hora e dia da semana
# Mas antes precisamos converter as colunas de datas para datetime
datesCols = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date",
            "order_delivered_customer_date", "order_estimated_delivery_date", "shipping_limit_date",
            "review_creation_date", "review_answer_timestamp"]

for col in datesCols:
    data[col] = pd.to_datetime(data[col])

In [ ]:
# Orders by hour
orders_df["order_purchase_timestamp"] = pd.to_datetime(orders_df["order_purchase_timestamp"])
orderbyhour = orders_df.groupby(orders_df["order_purchase_timestamp"].dt.hour)["order_id"].count().reset_index().sort_values(by="order_purchase_timestamp", ascending=False)
orderbyhour.rename(columns={"order_id":"Total Orders", "order_purchase_timestamp": "Hour of Day"}, inplace=True)

In [ ]:
fig = px.bar(orderbyhour, x='Hour of Day', y='Total Orders', title='Número de pedidos por hora do dia')
fig.update_xaxes(title='Hora do dia')
fig.update_yaxes(title='Número total de pedidos')
fig.show()


Os pedidos começam a aumentar por volta das 6h da manhã e atingem o pico às 4 da tarde.

In [ ]:
# Orders by day of the week
orderbydow = data.groupby(data["order_purchase_timestamp"].dt.day_name())["order_id"].count().reset_index()
orderbydow.rename(columns={"order_id":"Total Orders", "order_purchase_timestamp": "Weekday Name"}, inplace=True)
orderbydow = orderbydow.sort_values(by="Total Orders", ascending=False)

In [ ]:
fig = px.bar(orderbydow, x='Weekday Name', y='Total Orders', title='Número de pedidos por dia da semana')
fig.update_xaxes(title='Dia da semana')
fig.update_yaxes(title='Número total de pedidos')
fig.show()


In [ ]:
# Define a ordem dos dias da semana
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Converte a coluna "Weekday Name" em uma categoria ordenada
weekday = pd.Categorical(orderbydow['Weekday Name'], categories=weekday_order, ordered=True)

# Cria um novo dataframe ordenado pela coluna de categoria "weekday"
orderbydow_ordered = orderbydow.assign(weekday=weekday).sort_values('weekday')

# Cria um gráfico de barras com os dias da semana em ordem
fig = px.bar(orderbydow_ordered, x='weekday', y='Total Orders', title='Número de pedidos por dia da semana')
fig.update_xaxes(title='Dia da semana')
fig.update_yaxes(title='Número total de pedidos')
fig.show()


Os pedidos atingem o pico no início da semana (segunda e terça-feira) e começam a declinar um pouco depois. Durante o final de semana, observa-se uma diminuição acentuada dos pedidos.

<h2 style="color: #6c5b7b">Como os produtos são avaliados?</h2>


<p>De que adianta vender muito, entregar rápido, se as notas dos clientes estiverem baixas? Isso queima o filme de qualquer negócio! Vamos entrar mais a fundo na questão.</p>



In [ ]:
reviewsocres = data.groupby("product_category_name")["review_score"].agg(["mean", "count"]).sort_values(by="mean",ascending=False)
bestrated = reviewsocres[reviewsocres["count"]>=30][:10]

fig = go.Figure(go.Bar(
            x=bestrated['mean'],
            y=bestrated.index,
            orientation='h'))

fig.update_layout(
    title='Produtos com as melhores avaliações',
    xaxis_title='Avaliação média',
    yaxis_title='Categoria do produto',
    height=600,
    width=800,
    margin=dict(l=100, r=20, t=50, b=50),
)

fig.show()


In [ ]:
reviewsocres = data.groupby("product_category_name")["review_score"].agg(["mean", "count"]).sort_values(by="mean",ascending=False)
worstrated = reviewsocres[reviewsocres["count"]>=30].sort_values(by='mean')[:10]

fig = go.Figure(go.Bar(
            x=worstrated['mean'],
            y=worstrated.index,
            orientation='h'))

fig.update_layout(
    title='Produtos com as piores avaliações',
    xaxis_title='Avaliação média',
    yaxis_title='Categoria do produto',
    height=600,
    width=800,
    margin=dict(l=100, r=20, t=50, b=50),
)

fig.show()


<h2 style="color: #6c5b7b">O método de pagamento afeta o status do pedido?</h2>


<p>.</p>



In [ ]:
# Por exemplo: o pagamento em dinheiro aumenta o cancelamento de pedidos?
cashvscancel = pd.crosstab(data["payment_type"], data["order_status"])
cashvscancel = cashvscancel[["canceled", "delivered"]]
cashvscancel["% Canceled"] = (cashvscancel["canceled"] / cashvscancel["delivered"] ) * 100
cashvscancel["Avg Cancelation Rate"] = (len(data[data["order_status"] == "canceled"]) / len(data[data["order_status"] == "delivered"])) * 100
cashvscancel


Podemos ver que a taxa de cancelamento é praticamente a mesma em todos os métodos de pagamento. Mas notamos uma leve desvio acima da média para o cartão de crédito.

<h2 style="color: #6c5b7b">Existe alguma relação entre o tempo de entrega e as pontuações das avaliações?</h2>


<p>.</p>



In [ ]:
# Adicionando uma coluna delta que calcula o tempo que levou para o pedido ser entregue
data["TimeToDeliveryinHours"] = (data["order_delivered_customer_date"] - data["order_purchase_timestamp"])
data["TimeToDeliveryinHours"] = data["TimeToDeliveryinHours"].apply(lambda x: x.total_seconds())
data["TimeToDeliveryinHours"] = round((data["TimeToDeliveryinHours"] / 3600) / 24, 2)
data.rename(columns={"TimeToDeliveryinHours" : "TimeToDeliveryinDays"}, inplace=True)

In [ ]:
# Principais estatísticas do tempo de entrega
data[["TimeToDeliveryinDays"]].describe()

O tempo médio de entrega é relativamente alto (12,44 dias), com mediana de (10,19 dias). Observamos um valor extremo atípico de 208 dias.

O boxplot é um gráfico que mostra como um conjunto de dados está distribuído, destacando informações como a mediana (valor que divide os dados em duas partes iguais), os valores mínimo e máximo e a presença de dados incomuns. Ele é útil para identificar a forma como os dados estão agrupados e se existem pontos que se destacam do restante do conjunto.

In [ ]:

# Define a ordem crescente dos valores da pontuação de avaliação
score_order = sorted(data['review_score'].unique())

# Converte a coluna "review_score" em uma categoria ordenada
score = pd.Categorical(data['review_score'], categories=score_order, ordered=True)

# Cria um novo dataframe ordenado pela coluna de categoria "score"
data_ordered = data.assign(score=score).sort_values('score')

# Cria um seletor de pontuação de avaliação em ordem crescente
fig = px.box(data_ordered, x='score', y='TimeToDeliveryinDays', color='score',
             title='Relação entre a pontuação da avaliação e o tempo de entrega')

fig.update_xaxes(title='Pontuação da avaliação')
fig.update_yaxes(title='Tempo de entrega (dias)')

fig.show()


Podemos ver que temos vários valores atípicos. Estes são pedidos que demoraram muito para serem entregues por algum motivo.

In [ ]:
# Define o limite superior dos valores
q_high = data["TimeToDeliveryinDays"].quantile(0.95)

# Cria um novo dataframe sem os valores atípicos
data_no_outliers = data[data["TimeToDeliveryinDays"] < q_high]

# Define a ordem crescente dos valores da pontuação de avaliação
score_order = sorted(data_no_outliers['review_score'].unique())

# Converte a coluna "review_score" em uma categoria ordenada
score = pd.Categorical(data_no_outliers['review_score'], categories=score_order, ordered=True)

# Cria um novo dataframe com a coluna de categoria "score"
data_score = data_no_outliers.assign(score=score)

data_score = data_score.assign(score=score).sort_values('score')

# Cria um seletor de pontuação de avaliação em ordem crescente
fig = px.box(data_score, x='score', y='TimeToDeliveryinDays', color='score',
             title='Relação entre a pontuação da avaliação e o tempo de entrega')

fig.update_xaxes(title='Pontuação da avaliação', categoryorder='array', categoryarray=score_order)
fig.update_yaxes(title='Tempo de entrega (dias)')

fig.show()


Nossa hipótese inicial é confirmada. Podemos claramente observar uma relação direta entre o tempo de entrega e a pontuação da avaliação. À medida que o tempo de entrega diminui, a pontuação da avaliação tende a aumentar.

<h2 style="color: #6c5b7b">Quais são as cidades dos vendedores com menor/maior tempo de entrega?</h2>


<p>.</p>



In [ ]:
# Vendedores com melhor tempo de entrega
sellersdeliverytime = data.groupby("seller_city")["TimeToDeliveryinDays"].agg(["min", "max", "mean", "std", "count" ]).dropna().sort_values("mean").reset_index()
# Filtro para vendedores com 30 ou mais pedidos em seu histórico
sellersdeliverytime = sellersdeliverytime[sellersdeliverytime["count"]>=30]

In [ ]:
fastestdeliverysellers = sellersdeliverytime[:10]
slowestdeliverysellers = sellersdeliverytime.sort_values("mean", ascending=False)[:10]

In [ ]:
# Vendedores com entrega mais rápida.
fastestdeliverysellers

In [ ]:
# Vendedores com entrega mais lenta.
slowestdeliverysellers

In [ ]:
# Junte a pontuação média de avaliação à tabela acima.
avg_review_score_seller = data.groupby("seller_city")["review_score"].mean().dropna().sort_values(ascending=False).reset_index()

In [ ]:
sellerPerf = sellersdeliverytime.merge(avg_review_score_seller, on="seller_city")

In [ ]:
# Gráfico de regressão linear entre a média de tempo de entrega e a média da pontuação de avaliação
fig = px.scatter(sellerPerf, x='mean', y='review_score', trendline='ols',
                 title='Relação entre a média de tempo de entrega e a média da pontuação de avaliação')

fig.update_xaxes(title='Média de tempo de entrega (dias)')
fig.update_yaxes(title='Média da pontuação de avaliação')

fig.show()


<h2 style="color: #6c5b7b">Estados com maior/menor tempo de entrega.</h2>


<p>.</p>



In [ ]:
# Top 10 estados com o maior tempo médio de entrega
highestTTDstates = data.groupby("customer_state")["TimeToDeliveryinDays"].mean().dropna().sort_values(ascending=False).reset_index()
highestTTDstates = highestTTDstates[:10]

# Gráfico de barras com o tempo médio de entrega por estado
fig = px.bar(highestTTDstates, x='TimeToDeliveryinDays', y='customer_state',
             orientation='h', title='Top 10 estados com o maior tempo médio de entrega')

fig.update_yaxes(title='Estado')
fig.update_xaxes(title='Tempo médio de entrega (dias)')

fig.show()


In [ ]:
# Top 10 estados com o menor tempo médio de entrega
lowestTTDstates = data.groupby("customer_state")["TimeToDeliveryinDays"].mean().dropna().sort_values(ascending=True).reset_index()
lowestTTDstates = lowestTTDstates[:10]

# Gráfico de barras com o tempo médio de entrega por estado
fig = px.bar(lowestTTDstates, x='TimeToDeliveryinDays', y='customer_state',
             orientation='h', title='Top 10 estados com o menor tempo médio de entrega')

fig.update_yaxes(title='Estado')
fig.update_xaxes(title='Tempo médio de entrega (dias)')

fig.show()



<h2 style="color: #6c5b7b">Como o tempo médio de entrega varia ao longo do tempo?</h2>


<p>.</p>



In [ ]:
# Mediana do tempo de entrega por ano
deliverytimevstime = data.groupby(data["order_purchase_timestamp"].dt.year)["TimeToDeliveryinDays"].median().dropna()

# Converte ano em categoria (e não um número)
deliverytimevstime.index = deliverytimevstime.index.astype(str)

# Gráfico de barras com a mediana do tempo de entrega por ano
fig = px.bar(deliverytimevstime, x=deliverytimevstime.index, y='TimeToDeliveryinDays',
             title='Mediana do tempo de entrega por ano')

fig.update_xaxes(title='Ano')
fig.update_yaxes(title='Mediana de entrega em dias')

fig.show()


Podemos ver que a empresa fez uma melhoria significativa no tempo de entrega. Em 2016, o tempo médio de entrega excedia 17 dias, enquanto em 2017 ele caiu para cerca de 12 dias e para cerca de 9 dias em 2018.

<h2 style="color: #6c5b7b">Como a pontuação média das avaliações varia ao longo do tempo?</h2>


<p>.</p>



In [ ]:
# Média da pontuação da avaliação por ano
scorevstime = data.groupby(data["order_purchase_timestamp"].dt.year)["review_score"].mean().dropna()

# Converte o índice do grupo em uma categoria
scorevstime.index = scorevstime.index.astype(str)

# Gráfico de barras com a média da pontuação da avaliação por ano
fig = px.bar(scorevstime, x=scorevstime.index, y='review_score',
             title='Média da pontuação da avaliação por ano')

fig.update_xaxes(title='Ano')
fig.update_yaxes(title='Pontuação média da avaliação')

fig.show()


Os clientes estão dando pontuações melhores em 2017 e 2018 do que em 2016.

<h2 style="color: #6c5b7b">Quais são as categorias de produtos mais vendidas?</h2>


<p>.</p>


In [ ]:
top_categ_by_revenue = data.groupby("product_category_name").agg({'order_id':'nunique','payment_value':'sum'}).sort_values("payment_value", ascending=False)[:10]
top_categ_by_revenue.rename(columns={"order_id":"NumOfOrders", "payment_value":"Revenues"}, inplace=True)

In [ ]:
top_categ_by_revenue

<p>Vamos fazer um mapa coroplético com o tempo de entrega em dias por cada unidade federativa?</p>


Para fazer isso, precisamos de um arquivo que tenha as delimitações dos estados brasileiros em forma de polígonos. Podemos encontrar esse arquivo [aqui](https://github.com/fititnt/gis-dataset-brasil/tree/master/uf/geojson)

<p>Para ler o arquivo, utilizaremos o <strong>Geopandas</strong>! É uma biblioteca Python que estende o poder do Pandas para permitir a manipulação de dados geoespaciais. </p>

In [ ]:
# Agrupar os dados por estado e obter a média do tempo de entrega
mean_delivery_time_by_state = data.groupby('customer_state')['TimeToDeliveryinDays'].mean().reset_index()

mean_delivery_time_by_state

In [ ]:
import geopandas as gpd

geojson_file = './data/raw/uf.json'
geojson_data = gpd.read_file(geojson_file)

In [ ]:
geojson_data.head(10)

In [ ]:
geojson_data['tempo_entrega'] = mean_delivery_time_by_state['TimeToDeliveryinDays']

geojson_data.head()

In [ ]:
fig = px.choropleth_mapbox(geojson_data,
                           geojson=geojson_data.geometry,
                           locations=geojson_data.index,
                           color='tempo_entrega',
                           color_continuous_scale='YlOrRd',
                           mapbox_style='open-street-map',
                           zoom=3, center={'lat': -15.788497, 'lon': -47.879873},
                           opacity=0.5)

fig.show()
